In [3]:
from nuscenes.nuscenes import NuScenes
from nuscenes.map_expansion.map_api import NuScenesMap
from nuscenes.eval.prediction.splits import get_prediction_challenge_split
from nuscenes.prediction import PredictHelper
from nuscenes.utils.splits import create_splits_scenes


DATA_PATH = "../data/v1.0-mini/" #path to data stored

nusc = NuScenes(version='v1.0-mini', dataroot=DATA_PATH, verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.5 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [42]:
import json
from collections import Counter
NUM_AGENTS = 2

In [21]:
#         self.trainset = get_prediction_challenge_split("mini_train", dataroot=data_path)
helper = PredictHelper(nusc)

#get all the scenes
scenes = create_splits_scenes()
print(scenes.keys())
#get all the scenes in the trainset
set_name = "mini_train"
trainset = scenes[set_name] #List of scenes as part of training set
prediction_scenes = json.load(open(DATA_PATH+"maps/prediction_scenes.json", "r")) #Dictionary containing list of instance and sample tokens for each scene
 
print("Number of samples in train set: %d" % (len(trainset)))

dict_keys(['train', 'val', 'test', 'mini_train', 'mini_val', 'train_detect', 'train_track'])
Number of samples in train set: 8


In [57]:
idx = 1
scene = trainset[idx]
scene_tokens = prediction_scenes[scene] 

# print(scene_tokens)
if len(scene_tokens) < 2:
    print("Not enough agents in the scene")
    
else:
    #get the tokens in the scene: we will be using the instance tokens as that is the agent in the scene
    tokens = [scene_tok.split("_") for scene_tok in scene_tokens]
    instance_tokens, sample_tokens = list(list(zip(*tokens))[0]), list(list(zip(*tokens))[1]) #List of instance tokens and sample tokens
    
    tokenCount = Counter(instance_tokens) #Dictionary containing count for number of samples per token
    minCount = sorted(list(tokenCount.values()), reverse=True)[NUM_AGENTS-1] #used to find n agents with highest number of sample_tokens
    
    #Convert isntance and sample tokens to dict format
    instance_sample_tokens = {}
    for instance_token, sample_token in zip(instance_tokens, sample_tokens):
        if tokenCount[instance_token] >= minCount:
            try:
                instance_sample_tokens[instance_token].append(sample_token)
            except:
                instance_sample_tokens[instance_token] = [sample_token]
    
    
    print(len(instance_sample_tokens.keys()))
    print(instance_sample_tokens.keys())
#     print(instance_tokens)
#     print("---------------")
#     print(sample_tokens)

    # print(scenes["mini_train"])

2
dict_keys(['2e19253f3bac458191cf64ff2b22ba2b', 'd9a4df9e92804a44a039eb21e0dc1349'])


In [53]:
a = [1, 23, 4 , 5]
b = [2, 3, -4 , -5]
a.sort(reverse=True)
print(a)
# for aa,bb in zip(a, b):
#     print(aa, "     ", bb)

[23, 5, 4, 1]
